<h1 align=center><font size = 5>Part 1 - Scrapping web data and cleaning it
</font></h1>

### First, let install the necessary libraries

In [4]:
!pip install beautifulsoup4

In [5]:
!pip install lxml

In [6]:
!pip install html5lib

### Now lets imprt some useful libraries

In [7]:
from bs4 import BeautifulSoup

In [8]:
import requests

### Now we will assign the url from where we want to extract the table to a variable called website_url

In [9]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

### Using soup, we can get the html files of the webiste from where we want to extract the table

In [10]:
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

### Now we can extract the table from the html file and name it My_table

In [11]:
My_table = soup.find('table',{'class':'wikitable sortable'})

### Now we will extract the column named Postcode

In [12]:
Postcode = []

for row in My_table.findAll("tr"): #tr is one row
    cells = row.findAll("td") #td is the input in each cell
    if len(cells) == 3: #table has three columns
        Postcode.append(cells[0].find(text=True)) #Postcode

### we will also extract the column named Neighborhood

In [13]:
Neighborhood = []
for row in My_table.findAll("tr"): #tr is one row
    cells = row.findAll("td") #td is the input in each cell
    if len(cells) == 3: #table has three columns
        Neighborhood.append(cells[2].findAll(text=True))

### Similarly, we will extract the column named Borough

In [14]:
Borough = []
for row in My_table.findAll("tr"): #tr is one row
    cells = row.findAll("td") #td is the input in each cell
    if len(cells) == 3: #table has three columns
        Borough.append(cells[1].find(text=True))


### Next, let's put the extracted data into a proper dataframe

In [15]:
import pandas as pd
df = pd.DataFrame()
df['Postcode'] = Postcode

In [16]:
df['Borough'] = Borough

In [17]:
df['Neighborhood'] = Neighborhood

### Let's view the data. 

In [18]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,[Not assigned ]
1,M2A,Not assigned,[Not assigned ]
2,M3A,North York,"[Parkwoods, ]"
3,M4A,North York,"[Victoria Village, ]"
4,M5A,Downtown Toronto,"[Harbourfront, ]"


In [19]:
type(df['Neighborhood'][0]) #check the type of the values in the Neighborhood column

bs4.element.ResultSet

### checking the type of the Neighborhood column above, we realize that they are not strings, and hence we need to convert them strings

In [20]:
df['Neighborhood'] = df['Neighborhood'].astype(str) # convert the Neighborhood olumn to string

In [21]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,['Not assigned\n']
1,M2A,Not assigned,['Not assigned\n']
2,M3A,North York,"['Parkwoods', '\n']"
3,M4A,North York,"['Victoria Village', '\n']"
4,M5A,Downtown Toronto,"['Harbourfront', '\n']"


### In the following codes, we further clean the data, especially the Neighborhood column. First, let's remove unecessary characters

In [22]:
df.Neighborhood = df.Neighborhood.str[:-3] #remove the last three characters

In [23]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,['Not assigned\
1,M2A,Not assigned,['Not assigned\
2,M3A,North York,"['Parkwoods', '\"
3,M4A,North York,"['Victoria Village', '\"
4,M5A,Downtown Toronto,"['Harbourfront', '\"


In [24]:
import string
df.Neighborhood = df.Neighborhood.apply(lambda x: x.translate(str.maketrans('','',string.punctuation))) #further remove unecessary parts of the column

In [25]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### convert the Borough column to string

In [26]:
df['Borough'] = df['Borough'].astype(str) # convert the Borough column to string

### convert the Postcode column to string

In [27]:
df['Postcode'] = df['Postcode'].astype(str) # convert the Postcode column to string

In [28]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Remove white spaces

In [29]:
#Remove white spaces
df['Borough'] = df['Borough'].apply(lambda x: x.strip())
df['Postcode'] = df['Postcode'].apply(lambda x: x.strip())
df.Neighborhood = df.Neighborhood.apply(lambda x: x.strip())

In [30]:
df.head(6)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


### Copy the data into another dataframe so that we keep the original data

In [31]:
#copy data
df2 = df.copy()

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is  listed twice and has two neighborhoods: Harbourfront and Regent Park. We will combined two rows into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [32]:
df2 =df2.groupby('Postcode')['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index() #GROUPING DATA BY Postcode


Checking the results if we got the right results

In [33]:
df2.loc[df2['Postcode'] == 'M5A'] #checking to see if code works

,Postcode,Neighborhood
80,M5A,"Harbourfront, Regent Park"


In [34]:
len(df2)

180

### Let's merge the data - df2 and the original data so that we could add the Borough column

In [35]:
#merge data
df3 = df2.merge(df, left_on='Postcode', right_on='Postcode', how = 'inner')

In [36]:
df3.loc[df3['Postcode'] == 'M5A'] #checking to see if code works

,Postcode,Neighborhood_x,Borough,Neighborhood_y
118,M5A,"Harbourfront, Regent Park",Downtown Toronto,Harbourfront
119,M5A,"Harbourfront, Regent Park",Downtown Toronto,Regent Park


### Next, we will remove unecessary duplicates

In [37]:
df4 = df3[~df3[['Postcode', 'Neighborhood_x']].apply(frozenset, axis=1).duplicated()] #remove duplicates

In [38]:
df4.head()

,Postcode,Neighborhood_x,Borough,Neighborhood_y
0,M1A,Not assigned,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough,Rouge
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,Highland Creek
6,M1E,"Guildwood, Morningside, West Hill",Scarborough,Guildwood
9,M1G,Woburn,Scarborough,Woburn


In [39]:
df4.loc[df4['Postcode'] == 'M5V'] #checking to see if code works

,Postcode,Neighborhood_x,Borough,Neighborhood_y
148,M5V,"CN Tower, Bathurst Quay, Island airport, Harbo...",Downtown Toronto,CN Tower


### We will drop the Neighborhood_y column as we do not need it.

In [40]:
#remove or drop Neighborhood_y column
df4 = df4.drop(columns='Neighborhood_y')


### Rename Neighborhood_X as Neighborhood

In [41]:
#Rename Neighborhood_X
df4.rename(columns={'Neighborhood_x':'Neighborhood'}, inplace=True)

In [42]:
df4.head()

,Postcode,Neighborhood,Borough
0,M1A,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
6,M1E,"Guildwood, Morningside, West Hill",Scarborough
9,M1G,Woburn,Scarborough


### Now we will Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [43]:
#Now we will Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df5 = df4.loc[df4['Borough'] != 'Not assigned'] 

In [44]:
df5.head()

,Postcode,Neighborhood,Borough
1,M1B,"Rouge, Malvern",Scarborough
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
6,M1E,"Guildwood, Morningside, West Hill",Scarborough
9,M1G,Woburn,Scarborough
10,M1H,Cedarbrae,Scarborough


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park. We will do that with the following codes

In [45]:
import numpy as np

#Add a column call Neighborhood2
df5['Neighborhood2'] = np.where(df5['Neighborhood'] == 'Not assigned', df5.Borough, df5.Neighborhood)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [46]:
df5.head()

,Postcode,Neighborhood,Borough,Neighborhood2
1,M1B,"Rouge, Malvern",Scarborough,"Rouge, Malvern"
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,"Highland Creek, Rouge Hill, Port Union"
6,M1E,"Guildwood, Morningside, West Hill",Scarborough,"Guildwood, Morningside, West Hill"
9,M1G,Woburn,Scarborough,Woburn
10,M1H,Cedarbrae,Scarborough,Cedarbrae


### we will check to see if our code works

In [47]:
# check to see if it works
df5.loc[df5['Neighborhood'] == 'Not assigned'] #checking to see if code works

,Postcode,Neighborhood,Borough,Neighborhood2
194,M7A,Not assigned,Queen's Park,Queen's Park


### Now we can drop the Neighborhood column

In [48]:
# Now we can drop the Neighborhood column
df5 = df5.drop(columns='Neighborhood')

### Rename the Neighborhood2 coloumn back as Neighborhood

In [49]:
# Rename the Neighborhood2 coloumn back as Neighborhood
df5.rename(columns={'Neighborhood2':'Neighborhood'}, inplace=True)

In [50]:
#Rename final data
finaldata  = df5

In [51]:
finaldata.head()

,Postcode,Borough,Neighborhood
1,M1B,Scarborough,"Rouge, Malvern"
3,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
6,M1E,Scarborough,"Guildwood, Morningside, West Hill"
9,M1G,Scarborough,Woburn
10,M1H,Scarborough,Cedarbrae


### checking the shape of the finaldata

In [52]:
finaldata.shape

(103, 3)